In [1]:
import cv2
import json
import os
import sys

import numpy as np
import tensorflow as tf

sys.path.append('..')
import utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/sebas/Documentos/bootcamp/assignment1/.env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])

In [3]:
json_path = os.path.join(params['data_dir'], '..')
ground_truth_json = os.path.join(params['data_dir'],'activity_net.v1-3.min.json')
prediction_json = os.path.join(json_path, 'sample_detection_prediction.json')

with open(ground_truth_json, 'r') as file:
    ground_truth = json.load(file)
    
with open(prediction_json, 'r') as file:
    prediction = json.load(file)

In [4]:
def calc_union(min_1,max_1,min_2,max_2):
    return np.max([max_1,max_2]) - np.min([min_1,min_2])

def calc_overlap(min_1,max_1,min_2,max_2):
    return np.max([0,np.min([max_1,max_2])-np.max([min_1,min_2])])

def compute_iou(union,overlap):
    return union/overlap

def compute_AP(true_positive,false_positive):
    return true_positive/(true_positive+false_positive)

def compute_mAP(clases_ap):
    """
    HAY QUE VERIFICAR QUE EL NUMERO DE CLASES AQUI SEA <=101
    """
    return sum(clases_ap.values())/101

def compute_average_mAP(threshold_mAP):
    num_threshold = len(threshold_mAP)
    return sum(threshold_mAP.values())/num_threshold

def compute_classes_ap(clases_true_positive,clases_false_positive):
    clases_ap = {}
    for label in clases_true_positive:
        clases_ap[label] = compute_AP(
            clases_true_positive[label],
            clases_false_positive[label])
        
    return clases_ap

In [5]:
def compute_threshold_mAP():
    threshold_mAP = {}
    thresholds = np.linspace(0.5, 0.95, num=10)
    for threshold_iou in thresholds:
        clases_fp = {}
        clases_tp = {}
        for key,value in prediction['results'].items():
            ground_truth_key = ground_truth['database'][key]['annotations']
            for list_indice,predicted_segment in enumerate(prediction['results'][key]):

                true_positive = 0
                false_positive = 0
                predicted_label = predicted_segment['label']
                truth_label = ground_truth_key[list_indice]['label']

                if predicted_label == truth_label:

                    min_1 = predicted_segment['segment'][0]
                    max_1 = predicted_segment['segment'][1]

                    min_2 = ground_truth_key[list_indice]['segment'][0]
                    max_2 = ground_truth_key[list_indice]['segment'][1]

                    union = calc_union(min_1,max_1,min_2,max_2)
                    overlap = calc_overlap(min_1,max_1,min_2,max_2)
                    iou = compute_iou(union,overlap)

                    if iou >= threshold_iou:
                        true_positive = 1
                    else:
                        false_positive = 1

                else:
                    false_positive = 1

                if predicted_label in clases_tp.keys():
                    clases_tp[predicted_label] += true_positive
                    clases_fp[predicted_label] += false_positive
                else:
                    clases_tp[predicted_label] = true_positive
                    clases_fp[predicted_label] = false_positive


        clases_ap = compute_classes_ap(clases_tp,clases_fp)
        threshold_mAP[threshold_iou] = compute_mAP(clases_ap)
        
    return threshold_mAP

In [6]:
threshold_mAP = compute_threshold_mAP()
compute_average_mAP(threshold_mAP)

/home/sebas/Documentos/bootcamp/assignment1/.env/lib/python3.6/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in double_scalars
/home/sebas/Documentos/bootcamp/assignment1/.env/lib/python3.6/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in long_scalars


1.9791578931648819